In [1]:
import numpy as np
import cv2
from collections import namedtuple
from types import SimpleNamespace

In [2]:
# Implementation of "Three-Dimensinal Location Estimation of Circular Features for Machine Vision" -Reza Safaee-Rad .etc 

class Ellipse():
    def __init__(self, p=None, params=None, rect=None):
        """
        Args:
            p (None, optional): points to fit ellipse
            params (None, optional): ellipse parameters
            rect: parameters can be attained from captured frames
        """
        # a*x^2 + b*y^2  + 2*f*y + 2*g*x + 2*h*x*y + d = 0 
        if params != None:
            self.params = namedtuple("params", ('a','b','d','f','g','h'))(*params)
            self.rotate_rect = None 
        else:
            if rect == None:
                rotate_rect = cv2.fitEllipse(p)
                
                center = rotate_rect[0]
                radius = (rotate_rect[1][1], rotate_rect[1][0])
                angle = rotate_rect[2]+90

                self.rotate_rect = (center,radius,angle)
            else:
                self.rotate_rect = rect

            # https://en.wikipedia.org/wiki/Ellipse
            # The general equation's coefficients can be obtained from known semi-major axis a, semi-minor axis b, center coordinates xc yc,
            # and rotation angle theta  (the angle from the positive horizontal axis to the ellipse's major axis) using the formulae:
            xc, yc = self.rotate_rect[0]
            a,b = np.array(self.rotate_rect[1])/2
            theta = self.rotate_rect[2]/180.0*np.pi # Covert to rad

            # Ax^2 + Cy^2 + Bxy +  Dx + Ey + F = 0
            A = (a**2) * ((np.sin(theta))**2) + (b**2) * ((np.cos(theta))**2)
            B = 2*(b**2 - a**2)*np.sin(theta)*np.cos(theta)
            C = (a**2)*((np.cos(theta))**2) + (b**2)*((np.sin(theta))**2)
            D = -2*A*xc + (-1)*B*yc
            E = -1*B*xc + (-2)*C*yc
            F = A*(xc**2) + B*xc*yc + C*(yc**2) - (a**2)*(b**2)

            # a*x^2 + b*y^2  + 2*f*y + 2*g*x + 2*h*x*y + d = 0
            p = [A,C,F,E/2.0,D/2.0,B/2.0]
            self.params = namedtuple("params", ('a','b','d','f','g','h'))(*p)

    def get_points(self):
        xc, yc = self.rotate_rect[0]
        a,b = np.array(self.rotate_rect[1])/2
        theta = self.rotate_rect[2]/180.0*np.pi # Covert to rad

        s = np.linspace(0, 2*np.pi, 10000) # Centrifugality
        # https://blog.csdn.net/LaineGates/article/details/105745843
        r = a*np.cos(s)*np.cos(theta) - b*np.sin(s)*np.sin(theta) + xc
        c = a*np.cos(s)*np.sin(theta) + b*np.sin(s)*np.cos(theta) + yc
        return r,c

class Cone():
    def __init__(self, v=None, base=None, params=None, radius=None):
        """
        Args:
            v (None): 3d point vertex (x,y,z) in the ICS (position of camera center)
            base (TYPE): params of size 6 that defines an ellipse in the camera frame
            params (None, optional): cone parameters
            radius (None, optional): radius of circular feature 
        """
        # Cone parameters initilization from params or from base&&vertex
        self.vertex, self.radius = v, radius
        if params == None:
            # Base ellipse and vertex point
            self.base = base.params
            # https://stackoverflow.com/questions/16279212/how-to-use-dot-notation-for-dict-in-python
            self.params = SimpleNamespace(**(dict.fromkeys(['a','b','c','d','f','g','h','u','v','w'], None)))
            # Conic function parameters initialization
            bp = base.params # base params
            self.params.a = (v[2]**2)*bp.a
            self.params.b = (v[2]**2)*bp.b
            self.params.c = (v[0]**2)*bp.a + 2*bp.h*(v[0]*v[1]) + bp.b*(v[1]**2) + 2*bp.g*v[0]+2*bp.f*v[1] + bp.d
            self.params.d = (v[2]**2)*bp.d
            self.params.f = -1*v[2]*(bp.b*v[1]+bp.h*v[0]+bp.f)
            self.params.g = -1*v[2]*(bp.h*v[1]+bp.a*v[0]+bp.g)
            self.params.h = (v[2]**2)*bp.h
            self.params.u = (v[2]**2)*bp.g
            self.params.v = (v[2]**2)*bp.f
            self.params.w = -1*v[2]*(bp.f*v[1]+bp.g*v[0]+bp.d)
        else:
            self.params = params
            
        # Triple roots are sorted in descending order
        p = self.params
        self.roots = np.roots([1, -1*(p.a + p.b + p.c),
                    p.b*p.c + p.c*p.a + p.a*p.b - (p.f**2) - (p.g**2) - (p.h**2),
                    -1*(p.a*p.b*p.c + 2*p.f*p.g*p.h - p.a*(p.f**2) - p.b*(p.g**2) - p.c*(p.h**2))])
        self.roots = sorted(filter(lambda x: not isinstance(x, complex), self.roots), reverse=True)
        assert len(self.roots)==3, "Should have 3 roots"

        # Transformation matrix T0, T1, T2, T3(x2)
        # T1, T0: Rotation and Translation from image frame to camera frame
        # T3, T2: Rotation and Translation from XYZ(canonical) frame to image frame 
        self.T0 = np.identity(4,dtype=np.float64)
        self.T0[2,3] = -1*self.vertex[2] # z' = z + delta(z)
        self.T1 = np.identity(4,dtype=np.float64)
        self.T2 = np.identity(4,dtype=np.float64)
        self.T3 = [np.identity(4,dtype=np.float64),np.identity(4,dtype=np.float64)]
        for i, r in enumerate(self.roots):
            t1 = (p.b - r)*p.g - p.f*p.h
            t2 = (p.a - r)*p.f - p.g*p.h
            t3 = -1*(p.a-r)*(t1/t2)/p.g - p.h/p.g
            m = 1/np.sqrt(1+(t1/t2)**2 + t3**2)
            l = (t1/t2)*m
            n = t3*m
            self.T1[:3,i] = np.array([l,m,n])
            self.T2[i,3] = -1*(p.u*l + p.v*m + p.w*n)/r

        # lambda 1 >= lambda 2 after sorting, (l,m,n)
        homo_norm = np.array(
            [
                np.sqrt(
                    (self.roots[0] - self.roots[1]) / (self.roots[0] - self.roots[2])
                    ),
                0,
                np.sqrt(
                    (self.roots[1] - self.roots[2]) / (self.roots[0] - self.roots[2])
                    ),
                1
            ],dtype=np.float64)
        
        # Double solutions for normal vector in canonical frame
        self.homo_norm = tuple([
                homo_norm, # first solution
                homo_norm*[-1,1,1,1] # second solution 
            ])
        
        # T3 also has double solution
        for i, n in enumerate(self.homo_norm):
            self.T3[i][:3,2] = n[:3]
            temp = np.linalg.norm(n[:2])
            self.T3[i][:2,0] = [-1*n[1]/temp, n[0]/temp]
            self.T3[i][:3,1] = [-1*n[0]*n[2]/temp, -1*n[1]*n[2]/temp, temp]

        norm1 = np.dot(self.T1, self.homo_norm[0])[:3]
        norm2 = np.dot(self.T1, self.homo_norm[1])[:3]

        self.norm = tuple([
            namedtuple('norm', ('l', 'm', 'n'))(*norm1), # First solution
            namedtuple('norm', ('l', 'm', 'n'))(*norm2)  # Second solution
        ])

        if radius != None:
            self.position = [self.get_position(0), self.get_position(1)]

    def get_position(self, idx):
        # Get A,B,C,D according to Transformation T3
        A,B,C,D = 0,0,0,0
        for i in range(3):
            A += self.roots[i] * (self.T3[idx][i,0]**2)
            B += self.roots[i] * self.T3[idx][i,0]*self.T3[idx][i,2]
            C += self.roots[i] * self.T3[idx][i,1]*self.T3[idx][i,2]
            D += self.roots[i] * (self.T3[idx][i,2]**2)

        # Always use a positive z
        Z0 = A * self.radius / np.sqrt(np.square(B)+np.square(C)-A*D)
        X0 = B/A*Z0*-1
        Y0 = C/A*Z0*-1

        position = np.array([X0,Y0,Z0,1],dtype=np.float64)
        position2 = np.array([-X0,-Y0,-Z0,1],dtype=np.float64)

        # T transformation from XYZ frame to camera frame
        T = self.T0.dot(self.T1).dot(self.T2).dot(self.T3[idx])
        T_inv = np.linalg.inv(T)
        position = T.dot(position)
        position2 = T.dot(position2)

        position = position if position[2]>=0 else position2 #Determine if Z is greater than 0

        return position

In [3]:
if __name__ == "__main__":

    # Check Solution for circular feature in 3D (same as "Experimental Results" in "Three-Dimensinal Location Estimation of Circular Features for Machine Vision")
    # a*x^2 + b*y^2  + 2*f*y + 2*g*x + 2*h*x*y + d = 0
    ellipse = Ellipse(params = [204.024, 225.0, 66.976, -177.452/2,-127.567/2,-102.452/2])
    cone = Cone([0,0,-1], ellipse,radius=4)
    
    cone_params = cone.params
    triple_roots = cone.roots
    normal_params = cone.homo_norm
    cone_normal = cone.norm
    cone_position = cone.position
    
    print("cone.params",cone_params)
    print("cone.roots",triple_roots)
    print("cone.homo_norm",normal_params)
    print("cone.norm",cone_normal)
    print("cone.position",cone_position)

cone.params namespace(a=204.024, b=225.0, c=66.976, d=66.976, f=-88.726, g=-63.7835, h=-51.226, u=-63.7835, v=-88.726, w=66.976)
cone.roots [274.28131576062276, 225.00011387698217, -3.281429637604916]
cone.homo_norm (array([0.42136655, 0.        , 0.90689042, 1.        ]), array([-0.42136655,  0.        ,  0.90689042,  1.        ]))
cone.norm (norm(l=0.15114494050327953, m=0.7382252981305817, n=0.6574029328808015), norm(l=0.5000002754899345, m=5.557581673198975e-07, n=0.866025244730011))
cone.position [array([11.9825488 , 13.33701693, 27.90162427,  1.        ]), array([11.82991919, 13.66000169, 27.81034878,  1.        ])]
